# Examples of Mathematical Programming Solvers in Python

* [SciPy](https://docs.scipy.org/doc/scipy/tutorial/optimize.html)
* [PuLP](https://coin-or.github.io/pulp/)
* [OR-Tools](https://developers.google.com/optimization)
* [Gurobi](https://www.gurobi.com/documentation/9.5/examples/python_examples.html)
* [PYTHON-MIP](https://www.python-mip.com/)
* [Pyomo](http://www.pyomo.org/)
* [GEKKO](https://gekko.readthedocs.io/en/latest/)
* [CVXPY](https://www.cvxpy.org/)
* [PyMathProg](https://pymprog.sourceforge.net/)
* [cvxopt](https://cvxopt.org/)
* [OptLang](https://optlang.readthedocs.io/en/latest/)
* [PICOS](https://picos-api.gitlab.io/picos/)
* [cylp](https://github.com/coin-or/cylp)

## Chess Set Problem

find $x_1$ and $x_2$ that maximize $5\times x_1 + 12\times *x2$

subject to

Machining time constraint: $x_1 + 2\times x_2 <= 160 $

Weight constraint: $x_1 + 3\times x_2 <= 200$

Non-negativity constraints: $x_1>0$ and $x_2>=0$

## Solving Chess Set Problem using SciPy

In [1]:
#!pip install scipy
import numpy as np
import scipy
from scipy.optimize import linprog

# declare coefficients of the objective function. profit maximization is converted into minimization problem as per SciPy requirement
c = -np.array([5,12])

lhs_constraints=([3,2], # machining time constraint
                 [1,3]) # weight constraint

rhs_constraints=([160, # machining time constraint
                  200]) # weight constraint

bounds = [(0, scipy.inf), (0, scipy.inf)]  # Bounds of the decision variables

results = linprog(c=c, A_ub=lhs_constraints, b_ub=rhs_constraints, bounds=bounds, method='highs-ds')


# print the results
print('LP Solution:')
print(f'Profit: = {-round(results.fun,2)} $')
print(f'Make {round(results.x[0],0)} small sets, and make {round(results.x[1],0)} large sets')

LP Solution:
Profit: = 811.43 $
Make 11.0 small sets, and make 63.0 large sets


## Solving Chess Set Problem using PuLP

In [2]:
#!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

# Define the model
model = LpProblem(name='ChessSet', sense=LpMaximize)

# Define the decision variables
# x = {i: LpVariable(name=f"x{i}", lowBound=0, upBound = None, cat='Integer') for i in range(1, 3)}
x1 = LpVariable('SmallSet', lowBound = 0, upBound =  None, cat='Integer')
x2 = LpVariable('LargeSet', lowBound = 0, upBound =  None, cat='Integer')

# Add constraints
model += (3*x1 + 2*x2 <=160, 'Machining time constraint')
model += (  x1 + 3*x2 <= 200, 'Weight constraint')

# Set the profit as the objective function
profit= 5*x1 + 12*x2
model.setObjective(profit)

# Solve the optimization problem
model.solve()

# print the results
print('LP Solution:')
print(f'Profit: = {model.objective.value()} $')    
print(f'Make {x1.value()} small sets, and make {x2.value()} large sets')

LP Solution:
Profit: = 811.0 $
Make 11.0 small sets, and make 63.0 large sets


## Solving Chess Set Problem using OR-Tools

In [3]:
#!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('GLOP')

# Define the decision variables
x1 = solver.NumVar(0, solver.infinity(), 'x1')
x2 = solver.NumVar(0, solver.infinity(), 'x2')

# Add constraints
ct = solver.Constraint(0, 160, 'ct')
ct.SetCoefficient(x1, 3)
ct.SetCoefficient(x2, 2)
ct = solver.Constraint(0, 200, 'ct')
ct.SetCoefficient(x1, 1)
ct.SetCoefficient(x2, 3)

# Set the profit as the objective function
objective = solver.Objective()
objective.SetCoefficient(x1, 5)
objective.SetCoefficient(x2, 12)
objective.SetMaximization()

# Solve the optimization problem
solver.Solve()

# print the results
print('LP Solution:')
print('Profit =', round(objective.Value(),2),'$')
print('Make', '%.1f'%round(x1.solution_value()),'small sets, and', '%.1f'%round(x2.solution_value()), 'large sets')


LP Solution:
Profit = 811.43 $
Make 11.0 small sets, and 63.0 large sets


## Solving Chess Set Problem using Gurobi

In [4]:
# !pip install gurobipy
from gurobipy import *

# Define the model
m = Model()

# Define the decision variables
x1 = m.addVar(name='x1')
x2 = m.addVar(name='x2')

# Set the profit as the objective function
m.setObjective(5*x1+12*x2, GRB.MAXIMIZE)

# Add constraints
m.addConstr(x1 >= 0)
m.addConstr(x2 >= 0)
m.addConstr(3*x1 + 2*x2 <= 160)
m.addConstr(x1 + 3*x2 <= 200)

# Solve the optimization problem
m.optimize()

# print the results
print('LP Solution:', round(m.objVal, 2))
result = []
for v in m.getVars():
    result.append('%.1f'%round(v.x))
print('Make', result[0],'small sets, and', result[1], 'large sets')

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD FX(tm)-8120 Eight-Core Processor, instruction set [SSE2|AVX]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x92feeb9d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6000000e+02   5.816417e+00   0.000000e+00      0s
       2    8.1142857e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.04 seconds (0.00 work units)
Optimal objective  8.114285714e+02
LP Solution: 811.43
Make 11.0 small sets, and 63.0 large sets


## Solving Chess Set Problem using PYTHON-MIP

In [5]:
#!pip install mip

from mip import *


m = Model(sense=MAXIMIZE, solver_name=CBC)

x1 = m.add_var(name='x1')
x2 = m.add_var(name='x2')

m += 3*x1 + 2*x2 <= 160
m += x1 + 3*x2 <= 200
m += x1 >= 0
m += x2 >= 0

m.objective = 5*x1+12*x2

status = m.optimize()
print('LP Solution:',m.objective_value,'$')
result = []
for v in m.vars:
       result.append('%.1f'%round(v.x))
print('Make', result[0],'small sets, and', result[1], 'large sets')


LP Solution: 811.4285714285713 $
Make 11.0 small sets, and 63.0 large sets


## Solving Chess Set Problem using Pyomo

In [6]:
# !pip install pyomo
from pyomo.environ import * 

model = ConcreteModel()

model.x1 = Var(domain=NonNegativeReals, name = 'x1')
model.x2 = Var(domain=NonNegativeReals, name = 'x2')

model.profit = Objective(expr = 5*model.x1 + 12*model.x2, sense=maximize)

model.noNeg1 = Constraint(expr = model.x1 >= 0)
model.noNeg2 = Constraint(expr = model.x2 >= 0)
model.machiningTime  = Constraint(expr = 3*model.x1 + 2*model.x2 <= 160)
model.weight = Constraint(expr = model.x1 + 3*model.x2 <= 200)

results = SolverFactory('gurobi').solve(model)
results.write()
if results.solver.status:
    model.pprint()

print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x1 = ', model.x1())
print('x2 = ', model.x2())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x3
  Lower bound: 811.4285714285714
  Upper bound: 811.4285714285714
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 3
  Number of nonzeros: 7
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved t

## Solving Chess Set Problem using GEKKO

In [7]:
# !pip install gekko
from gekko import GEKKO

m = GEKKO()

x1 = m.Var()
x2 = m.Var()

m.Equation(x1 >= 0)
m.Equation(x2 >= 0)
m.Equation(3*x1 + 2*x2 <= 160)
m.Equation(x1 + 3*x2 <= 200)

m.Maximize(5*x1 + 12*x2)

re = m.solve()

print('Make', x1.value,'small sets, and', x2.value, 'large sets')
print(type(x1.value))

apm 99.248.50.154_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            6
   Intermediates:            0
   Connections  :            0
   Equations    :            5
   Residuals    :            5
 
 Number of state variables:              6
 Number of total equations: -            4
 Number of slack variables: -            4
 ---------------------------------------
 Degrees of freedom       :             -2
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

************************************************************************

## Solving Chess Set Problem using CVXPY

In [8]:
# !pip install cvxpy
import cvxpy as cp
import numpy as np

# Define the decision variables
x1 = cp.Variable()
x2 = cp.Variable()

# Add constraints
constraints = [3*x1 + 2*x2 <= 160, x1 + 3*x2 <= 200, x1 >= 0, x2 >= 0]

# Set the profit as the objective function
obj = cp.Maximize(5*x1 + 12*x2)

# Solve the optimization problem
prob = cp.Problem(obj, constraints)

# print the results
print('LP Solution:')
print('Profit =', round(prob.solve(),2),'$')
print('Make', np.round(x1.value),'small sets, and', np.round(x2.value), 'large sets')

(CVXPY) Jan 25 06:16:54 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 25 06:16:54 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
LP Solution:
Profit = 811.43 $
Make 11.0 small sets, and 63.0 large sets


## Solving Chess Set Problem using PyMathProg

In [9]:
# !pip install pymprog
from pymprog import *

begin('model')

x1 = var('x1')
x1.kind = int
x2 = var('x2')
x2.kind = int

maximize(5*x1 + 12*x2, 'profit')

x1 >= 0
x2 >= 0
3*x1 + 2*x2 <= 160
x1 + 3*x2 <= 200

solve()
print('LP Solution:')
print('Profit =', round(vobj(),2),'$')
sensitivity()

LP Solution:
Profit = 811.0 $

PyMathProg 1.0 Sensitivity Report Created: 2023/01/25 Wed 18:16PM
Variable            Activity   Dual.Value     Obj.Coef   Range.From   Range.Till
--------------------------------------------------------------------------------
*x1                  11.4286            0            5            4           18
*x2                  62.8571            0           12      3.33333           15
Constraint       Activity Dual.Value  Lower.Bnd  Upper.Bnd RangeLower RangeUpper
--------------------------------------------------------------------------------
 R1                   160   0.428571       -inf        160    133.333        600
 R2                   200    3.71429       -inf        200    53.3333        240


## Solving Chess Set Problem using cvxopt

In [10]:
# !pip install cvxopt
from cvxopt.modeling import variable, op, max, sum

x1 = variable()
x2 = variable()

c1 = ( 3*x1+2*x2 <= 160 )
c2 = ( x1+3*x2 <= 200 )
c3 = ( x1 >= 0 )
c4 = ( x2 >= 0 )

lp1 = op(-5*x1-12*x2, [c1,c2,c3,c4])
lp1.solve()
print('Make', x1.value,'small sets, and', x2.value, 'large sets')

     pcost       dcost       gap    pres   dres   k/t
 0: -7.4247e+02 -1.5315e+03  2e+02  1e-02  8e-01  1e+00
 1: -8.1089e+02 -8.5162e+02  8e+00  6e-04  4e-02  5e-01
 2: -8.1139e+02 -8.1205e+02  1e-01  1e-05  7e-04  8e-03
 3: -8.1143e+02 -8.1143e+02  1e-03  1e-07  7e-06  8e-05
 4: -8.1143e+02 -8.1143e+02  1e-05  1e-09  7e-08  8e-07
Optimal solution found.
Make [ 1.14e+01]
 small sets, and [ 6.29e+01]
 large sets


## Solving Chess Set Problem using OptLang

In [11]:
# !pip install optlang
from optlang import Model, Variable, Constraint, Objective

x1 = Variable('x1', lb=0)
x2 = Variable('x2', lb=0)

c1 = Constraint(3 * x1 + 2 * x2, ub=160)
c2 = Constraint(x1 + 3 * x2, ub=200)

obj = Objective(5 * x1 + 12 * x2, direction='max')

model = Model(name='Simple model')
model.objective = obj
model.add([c1, c2])
status = model.optimize()

print("LP Solution:", round(model.objective.value,2),'$')
result = []
for var_name, var in model.variables.items():
    result.append('%.1f'%round(var.primal))
print('Make', result[0],'small sets, and', result[1], 'large sets')

LP Solution: 811.43 $
Make 11.0 small sets, and 63.0 large sets


## Solving Chess Set Problem using PICOS

In [12]:
# !pip install picos
import picos as pico

# Define the problem
prob = pico.Problem("ChessSet")


# Define the variables
x1 = pico.RealVariable("x1", lower = 0)
x2 = pico.RealVariable("x2", lower = 0)

# Define the constraints
prob.add_constraint(x1 >= 0)
prob.add_constraint(x2 >= 0)
prob.add_constraint(3*x1 + 2*x2 <= 160)
prob.add_constraint(x1 + 3*x2 <= 200)

# Define the objective function
prob.set_objective('max', 5*x1 + 12*x2)

# Solve the LP
prob.solve(solver='cvxopt')

# print the results
print('LP Solution:')
print(f'Profit: = {prob.value} $')
print(f'Make {round(x1,0)} small sets, and make {round(x2,0)} large sets')

LP Solution:
Profit: = 811.4285712416705 $
Make 11.0 small sets, and make 63.0 large sets


## Solving Chess Set Problem using cylp

In [13]:
# !pip install cylp

import cylp as cy 
from cylp.cy import CyClpSimplex

# Define the model
model = cy.py.modeling.CyLPModel()

# Define the variables
x1 = model.addVariable('x1', 1, isInt=True)
x2 = model.addVariable('x2', 1, isInt=True)

# Add the constraints
model += 3*x1[0]+2*x2[0]<=160
model += x1[0]+3*x2[0]<=200
model += x1[0] >= 0
model += x2[0] >= 0

# Define the objective function
model.objective = -1*(5*x1[0]+12*x2[0])

# The status of the solution
cbcModel = cy.cy.CyClpSimplex(model).getCbcModel()
print(cbcModel.solve())
print (cbcModel.status)

# print the results
ss=cbcModel.primalVariableSolution['x1'][0]
ls=cbcModel.primalVariableSolution['x2'][0]
print('LP Solution:')
print(f'Profit: = {-cbcModel.objectiveValue} $')
print(f'Make {round(ss,0)} small sets, and make {round(ls,0)} large sets')


0
solution
LP Solution:
Profit: = 811.0 $
Make 11.0 small sets, and make 63.0 large sets
